In [53]:
# Helpful source: https://en.wikipedia.org/wiki/Poker_probability#7-card_poker_hands
# http://people.math.sfu.ca/~alspach/comp20/

from math import comb

# TODO: introduce Card class and attempt the harder problem of probabilities for remaining cards.

NUM_CARDS = 52
NUM_SUITS = 4
CARDS_PER_SUIT = 13
ADJUSTED_CARDS_PER_SUIT = CARDS_PER_SUIT + 1 # Account for Ace being used at beginning and end when we need to.
CARDS_TO_CHOOSE = 7 # Hold 'em
CARDS_IN_HAND = 5

def distinct_hands():
    return comb(NUM_CARDS, CARDS_TO_CHOOSE)

print("Distinct hands:", distinct_hands())

def royal_flush_frequency():
    freq = NUM_SUITS 
    # For the other 2 cards, they can be any of the 47 not involved in our straight flush.
    freq = freq * comb(NUM_CARDS - CARDS_IN_HAND, 2)
    return freq

def straight_flush_frequency():
    # We shall count straight flushes using the largest card in the straight flush. This enables us to pick up 6- and 7-card straight flushes.
    # 5 through king can be the highest card, that's 9 cards
    num_highest_card = 9
    freq = 9 * NUM_SUITS
    # Now we need to account for the other 2 cards, which can be any cards except the straight successor, which means there are 46 possible choices rather than 41
    freq = freq * comb(NUM_CARDS - CARDS_IN_HAND  - 1, 2)
    return freq

def four_of_a_kind_frequency():
    # first let's account for the frequency of 4s, ignoring the 3 remaining cards.
    freq = CARDS_PER_SUIT
    # We can choose any 3 cards from the remaining 48
    freq = freq * comb(NUM_CARDS - 4, 3)
    return freq

def full_house_frequency():
    # There are 3 ways to get a full house and we count them separately, to avoid repeats. 
    # 1. Two triples and a singleton
    triplet_number_combos = comb(CARDS_PER_SUIT, 2) 
    suit_choices_for_triplet = comb(4,3)
    singleton = NUM_CARDS - (2 * NUM_SUITS)
    
    freq1 = triplet_number_combos * (suit_choices_for_triplet ** 2) * singleton

    # 2. A triple and two pairs
    triple_combos = CARDS_PER_SUIT
    pair_number_combos = comb(CARDS_PER_SUIT - 1, 2)
    suit_choices_for_pair = comb(4,2)
    freq2 = triple_combos * suit_choices_for_triplet * pair_number_combos * (suit_choices_for_pair ** 2)
    
    # 3. A triple, a pair, and 2 singletons
    triple_combos = CARDS_PER_SUIT
    pair_combos = CARDS_PER_SUIT - 1
    singleton_combos = comb(11, 2)
    suit_choices_for_singleton = 4
    freq3 = triple_combos * suit_choices_for_triplet * pair_combos * suit_choices_for_pair * singleton_combos * (suit_choices_for_singleton ** 2)
    
    return freq1 + freq2 + freq3


# Calculate the frequency of a straight (5 consecutive ranks) when we have 7 distinct ranks to choose from.  Note that we are not considering multiple suits here.  Used in both flush_frequency() and straight_frequency()
def seven_ranks_one_suit_straight_frequency():
    # There are 3 types of straights that appear with 7 distinct ranks:
    # 1. {x,x+1,x+2,x+3,x+4,x+5,x+6}
    seven_straight_freq = ADJUSTED_CARDS_PER_SUIT - (CARDS_TO_CHOOSE - 1)
    # 2. {x,x+1,x+2,x+3,x+4,x+5,y}, where y != x-1 | x+6.  We need to seperately consider the cases of x being Ace or 9, because y is free-er in those cases.
    six_straight_freq_a_or_9 = 2 * (CARDS_PER_SUIT - CARDS_TO_CHOOSE)
    six_straight_freq_other = 7 * (CARDS_PER_SUIT - CARDS_TO_CHOOSE - 1)
    six_straight_freq = six_straight_freq_a_or_9 + six_straight_freq_other
    # 3. {x,x+1,x+2,x+3,x+4,y,z}, where y and z != x-1 | x+5.  We need to separately consider the cases of x being Ace or 10, because y is free-er in those cases.
    five_straight_freq_a_or_10 = 2 * comb(CARDS_TO_CHOOSE, 2)
    five_straight_freq_other = 8 * comb(CARDS_TO_CHOOSE - 1, 2)
    five_straight_freq = five_straight_freq_a_or_10 + five_straight_freq_other

    return seven_straight_freq + six_straight_freq + five_straight_freq

# Calculate the frequency of a straight (5 consecutive ranks) when we have 6 distinct ranks to choose from.  Note that we are not considering multiple suits here.  Note also that the 7th card is not accounted for.  Used in both flush_frequency() and straight_frequency()
def six_ranks_one_suit_straight_frequency():
    # There are 2 types of straights that appear with 6 distinct ranks
    # 1. {x, x+1, x+2, x+3, x+4, x+5}
    six_straight_freq = ADJUSTED_CARDS_PER_SUIT - 6 + 1

    # 2. {x, x+1, x+2, x+3, x+4, y}, where y != x-1 | x+5. We need to separately consider the case s of x being Ace or 10, because y is free-er in those cases
    five_straight_freq_a_or_10 = 2 * (CARDS_PER_SUIT - 6)
    five_straight_freq_other = 8 * (CARDS_PER_SUIT - 6 - 1)
    five_straight_freq = five_straight_freq_a_or_10 + five_straight_freq_other

    return six_straight_freq + five_straight_freq

def flush_frequency():
    # A. Let's begin by calculating the non-straight 7-flushes
    one_suit_seven_flush_freq = comb(13, 7)
    one_suit_straight_freq = seven_ranks_one_suit_straight_frequency()
    one_suit_proper_seven_flush_freq = one_suit_seven_flush_freq  - one_suit_straight_freq
    proper_seven_flush_freq = NUM_SUITS * one_suit_proper_seven_flush_freq
    
    #B. Now let's calculate the non-straight 6-flushes
    one_suit_six_flush_freq = comb(13, 6) # Note that we are not initially considering the non-suited card.
    six_ranks_one_suit_straight_freq = six_ranks_one_suit_straight_frequency()
    one_suit_proper_six_flush_freq = one_suit_six_flush_freq - six_ranks_one_suit_straight_freq
    # Now we need to multiply out the free card, which can be any of the (39) cards from the other 3 suits.
    one_suit_proper_six_flush_freq = one_suit_proper_six_flush_freq * (NUM_CARDS - CARDS_PER_SUIT)
    proper_six_flush_freq = NUM_SUITS * one_suit_proper_six_flush_freq

    #C. Now let's calculate the remaining non-straight 5 flushes
    one_suit_five_flush_freq = comb(13, 5) # Note that we are not initially considering the two non-suited cards
    # We need to exclude five_straight
    five_straight_freq = ADJUSTED_CARDS_PER_SUIT - 5 + 1
    one_suit_proper_five_flush_freq = one_suit_five_flush_freq - five_straight_freq

    # Now we need to multiply out the free card, which can be chosen 2 from any of the (39) cards from the other 3 suits
    one_suit_proper_five_flush_freq = one_suit_proper_five_flush_freq * comb(NUM_CARDS - CARDS_PER_SUIT, 2)

    proper_five_flush_freq = NUM_SUITS * one_suit_proper_five_flush_freq

    return proper_seven_flush_freq + proper_six_flush_freq + proper_five_flush_freq

def straight_frequency():
    # A. 7 Distinct ranks
    seven_ranks_one_suit_straight_freq = seven_ranks_one_suit_straight_frequency()
    # 3 types of flushes to eliminate from 7 rank:
    # 1. 7-flush
    seven_ranks_seven_flush_freq = NUM_SUITS
    # 2. 6-flush
    seven_ranks_six_flush_freq = CARDS_TO_CHOOSE * NUM_SUITS * (NUM_SUITS - 1)
    # 3. 5-flush.
    seven_ranks_five_flush_freq = (comb(CARDS_TO_CHOOSE,5) * NUM_SUITS) * ((NUM_SUITS - 1) ** 2)
    
    seven_ranks_flush_freq = seven_ranks_seven_flush_freq + seven_ranks_six_flush_freq + seven_ranks_five_flush_freq

    all_suit_combos = NUM_SUITS ** CARDS_TO_CHOOSE
    # Remove the flush suit combos
    straight_non_flush_suit_combos = all_suit_combos - seven_ranks_flush_freq
    seven_ranks_straight_freq = seven_ranks_one_suit_straight_freq * straight_non_flush_suit_combos

    # B. 6 Distinct ranks (one pair)
    six_ranks_one_suit_straight_freq = six_ranks_one_suit_straight_frequency()
    # There are 6 choices for which rank will have a pair and there are 6 choices for a pair of that rank.
    six_ranks_straight_freq = six_ranks_one_suit_straight_freq * 6 * comb(4,2)
    remaining_5_card_suit_combos = 4 ** 5
    # 2 types of flushes to eliminate from the remaining 5 card suit combos:
    # 1. the rest of the cards form a flush
    five_same_suit_freq = NUM_SUITS
    # There are 5 ways to choose 4 cards to be in the same suit, 2 choices for that suit and 3 choices for the suit of the remaining card.
    four_matching_a_pair_suit_freq = comb(5, 4) * 2 * 3
    non_flush_suit_combos = remaining_5_card_suit_combos - five_same_suit_freq - four_matching_a_pair_suit_freq

    six_ranks_straight_freq = six_ranks_straight_freq * non_flush_suit_combos

    # C. 5 Distinct ranks (one three of a kind, OR 2 pairs)
    five_ranks_one_suit_straight_freq = ADJUSTED_CARDS_PER_SUIT - 5 + 1

    # 1. First we consider the triplet case
    # There are 5 choices for the rank of the trips, and 4 suit choices for trips of that rank.
    triplet_choices = 5 * NUM_SUITS
    remaining_4_card_choices = 4 ** NUM_SUITS
    # We need to subtract the 3 choices where those 4 cards are in the same suit as one of the three of a kinds
    remaining_4_card_choices = remaining_4_card_choices - 3

    five_ranks_with_triplet_freq = five_ranks_one_suit_straight_freq * triplet_choices * remaining_4_card_choices

    # 2. Now we consider the 2 pairs case
    # There are 5 choose 2 choses for the 2 ranks which will be paired
    pair_choices = comb(5, 2)
    # There are 6 suit choices for each of the pair choices, which gives 36 ways to choose the 2 pairs.  But rather than just directly multiply this 36, we have to break it down to consider the different flush possibilities
    # a) 6 of the ways of getting the 2 pairs have the same suits represented for the 2 pairs
    same_suits_pair_choices = comb(4,2)
    # b) 24 of them have exactly 1 suit in common between the 2 pairs, and
    one_common_suit_pair_choices = NUM_SUITS * (NUM_SUITS - 1) * (NUM_SUITS - 2)
    # c) 6 of them have no suit in common between the 2 pairs.
    diff_suits_pair_choices = comb(4,2)
    # Now let's consider these suit choices impact the remaining 3 cards' suit choices.
    remaining_suit_choices = NUM_SUITS ** 3
    # In case a), 2 of the overall suit choices must be eliminated because they would produce a (straight) flush
    same_suits_pair_choices = same_suits_pair_choices * (remaining_suit_choices - 2)
    # In case b), only 1 of the overall suit choices must be eliminated
    one_common_suit_pair_choices = one_common_suit_pair_choices * (remaining_suit_choices - 1)
    # In case c), a flush is impossible because there are not enough matching suits among the pairs to create one with the rest of the cards
    diff_suits_pair_choices = diff_suits_pair_choices * remaining_suit_choices

    five_ranks_with_with_pairs_freq = (pair_choices ** 2) * (same_suits_pair_choices + one_common_suit_pair_choices + diff_suits_pair_choices)

    five_ranks_straight_freq = five_ranks_with_triplet_freq + five_ranks_with_with_pairs_freq

    return seven_ranks_straight_freq + six_ranks_straight_freq + five_ranks_straight_freq

# CONTINYA HERE
def three_of_a_kind_frequency():
    pass

def two_pair_frequency():
    pass

def pair_frequency():
    pass

def high_card_frequency():
    pass

print("Royal Flush Frequency: ", royal_flush_frequency())
print("Straight Flush Frequency: ", straight_flush_frequency())
print("4 of a Kind Frequency: ", four_of_a_kind_frequency())
print("Full House Frequency: ", full_house_frequency())
print("Flush Frequency: ", flush_frequency())
print("Straight Frequency: ", straight_frequency())
#TODO: add other probabilities stuff we see on wiki




Distinct hands: 133784560
Royal Flush Frequency:  4324
Straight Flush Frequency:  37260
4 of a Kind Frequency:  224848
Full House Frequency:  3473184
Flush Frequency:  4047644
Straight Frequency:  6180020
